In [ ]:
import random
from copy import deepcopy

In [ ]:
perc_mut = 0.05

selected = []
for i in range(1000000):
	if random.random() < perc_mut:
		selected.append(True)

print((len(selected)/1000000) * 100)

In [ ]:
len(selected)

In [ ]:
ls = ["Hello", "world", "como", "estas"]
ls[ls.index("como")] = "how"



In [ ]:
def ls_ls(mut_ls):
	# deepcopy is required to prevent over writing of the original subnetwork list
	# -- mutations are done by index updating
	subnetworks = deepcopy(mut_ls)
	for ls in subnetworks:
		mut_network = []
		for idx, elm in enumerate(ls):
			if idx % 2 == 0:
				ls[ls.index(elm)] = "mutated"

	return subnetworks
		
		

In [ ]:
test_ls = [[1,2,3], [4,5,6], [10, 10, 10]]
new_ls = ls_ls(test_ls) 

In [ ]:
print(test_ls)

In [ ]:
new_ls

In [ ]:
for i in range(10000):
	if round(random.random(), 2) < 0.05:
		print("true")

In [ ]:
ds = {"net1":3, "net2":0, "net3":2}
print(type(list(ds.keys())[0]))

In [ ]:
def _dict_to_list(data):
	"""Converts dictioanry to string

	Parameters
	----------
	data : dict
		data containing string and int as key value pair types
	
	Returns
	-------
	list 
		list containing string repeated with their value

	Example
	-------
	>>> ds = {"net_1": 2, "net_2": 0, "net_3": 1}
	>>> ls = _dict_to_list(ds)
	>>> print(ls)
	>>> ["net_1", "net_1", "net_3"]
	"""
	# Type checking 
	if not isinstance(data, dict):
		raise ValueError("data must be dict, {} was given".format(type(data)))
	
	ls = []
	for key, value in data.items():
		key_list = [key for i in range(value)]
		
		# update list
		ls += key_list
	return ls
    

In [ ]:
_dict_to_list(ds)

In [ ]:
from collections import defaultdict

In [ ]:
test_dict = defaultdict(lambda: None) 
for idx, value in enumerate(["hello", "world", "k"]):
    test_dict[idx] = value

In [ ]:
test_dict

In [ ]:
x = test_dict[10]

In [ ]:
print(x)

In [ ]:
import random
from collections import Counter

In [ ]:
ls = "hello my name is erik serrano what is your name?".split()
choices = random.choices(ls, k=6)
print(choices)

In [ ]:
gene_name, counts = Counter(choices).most_common()[0]
print(gene_name)

dup_index = [i for i,value in enumerate(choices) if value == "name"]
print(dup_index)

In [ ]:
ls  = [0.33, 0.33, 0.33]
print(sum(ls))

In [ ]:

test = 0
for i in range(10):
	new_val = i
	test = i


In [ ]:
test 

In [ ]:
ls = [[1,2,3,4,5,6,7,8,9,10, 11, 12],
 [1*2,2*4,3*2,4*6,5*1,6*4,7*7,8*9,9*9,10*0, 11*1, 12*1],
]

ls1 = [1,2,3,4,5,6,7,8,9,10, 11, 12]


In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(ls1).describe().index.tolist()

In [1]:
import sys

from collections import defaultdict
from itertools import permutations
sys.path.append("../")
from simunet.common.parsers import parse_input, StringDB
from simunet.analysis.network import generate_fa_subnetworks

In [43]:
fa_data = parse_input("../Input/Input.gmt.txt")
networks = generate_fa_subnetworks(5000, fa_data)
db = StringDB("../db/STRING.txt")

In [15]:
# this the function
locus_and_score = defaultdict(list)
n_locus = len(fa_data.keys())
for locus_idx in range(1, n_locus+1):
    fa_key = "locus {}".format(locus_idx)
    gene_arr = set(fa_data[fa_key])
    print(locus_idx)
    gene_score = defaultdict(list)
    sub_dict = defaultdict(list)
    for network in networks[:10]:
        selected_gene = network[locus_idx-1]
        locus_genes = list(gene_arr - set(selected_gene))
        for gene in locus_genes:
            network[locus_idx-1] = gene
            gene_score =  db.targeted_gene_scoring(gene, locus_genes)
            sub_dict[gene].append(gene_score)
    locus_and_score[fa_key]  = sub_dict


        


1
2
3
4
5
6
7
8
9
10
11
12


In [22]:
scores = defaultdict(lambda: None)
for locus, genes_scores in locus_and_score.items():
    sub_dict = defaultdict(lambda: None)
    for gene, score_arr in genes_scores.items():
        score = round(sum(score_arr)/len(networks),3)
        sub_dict[gene] = score
    scores[locus] = sub_dict
        

In [50]:
def create_gmt_file(fa_data, networks, db):
    """write out a gmt file

    Parameters
    ----------
    fa_data : dict
        FA locus data
    networks : list
        list of networks
    db : dict
        stringDB
    """
    locus_and_score = defaultdict(list)
    n_locus = len(fa_data.keys())
    for locus_idx in range(1, n_locus+1):
        fa_key = "locus {}".format(locus_idx)
        gene_arr = set(fa_data[fa_key])
        print(locus_idx)
        gene_score = defaultdict(list)
        sub_dict = defaultdict(list)
        for network in networks[:10]:
            selected_gene = network[locus_idx-1]
            locus_genes = list(gene_arr - set(selected_gene))
            for gene in locus_genes:
                network[locus_idx-1] = gene
                gene_score =  db.targeted_gene_scoring(gene, locus_genes)
                sub_dict[gene].append(gene_score)
        locus_and_score[fa_key]  = sub_dict
        
    scores = defaultdict(lambda: None)
    for locus, genes_scores in locus_and_score.items():
        sub_dict = defaultdict(lambda: None)
        for gene, score_arr in genes_scores.items():
            score = round(sum(score_arr)/len(networks),3)
            sub_dict[gene] = score
        scores[locus] = sub_dict
        
    # next write gmt file
    outfile_data = []
    for locus_id, genes in fa_data.items():
        scored_arr = scores[locus_id]
        line_out = []
        line_out.append(locus_id)
        for gene in genes:
            score = scored_arr[gene] 
            if score == 0.0:
                score = "N/A"
                scored_gene = "{} {}".format(gene, score)
                line_out.append(scored_gene) 
            else:
                scored_gene = "{} {}".format(gene, score)
                line_out.append(scored_gene)
        outfile_data.append(line_out)

    # converting into a string
    with open("genescores.txt", "w") as outfile:
        for line_data in outfile_data:
            stdout = "\t".join(line_data)
            outfile.write("{}\n".format(stdout))

In [51]:
create_gmt_file(fa_data, networks, db)

1
2
3
4
5
6
7
8
9
10
11
12


In [37]:
outfile_data

[['locus 1',
  'NUPR1 N/A',
  'CTB-134H23.2 N/A',
  'SLC5A11 N/A',
  'KIAA0556 N/A',
  'CD19 N/A',
  'SH2B1 N/A',
  'CCDC101 N/A',
  'GTF3C1 N/A',
  'IL27 N/A',
  'ARHGAP17 N/A',
  'ERN2 N/A',
  'DCTN5 N/A',
  'NSMCE1 N/A',
  'AQP8 N/A',
  'RABEP2 N/A',
  'XPO6 N/A',
  'ATP2A1 N/A',
  'CHP2 N/A',
  'BOLA2 0.003',
  'KDM8 N/A',
  'EIF3C N/A',
  'ATXN2L N/A',
  'LAT N/A',
  'ZKSCAN2 N/A',
  'SULT1A1 N/A',
  'HS3ST4 N/A',
  'EIF3CL N/A',
  'TUFM N/A',
  'NPIPL1 N/A',
  'SNX29P2 N/A',
  'IL21R N/A',
  'PRKCB 0.001',
  'SPNS1 N/A',
  'TNRC6A N/A',
  'CACNG3 N/A',
  'PLK1 N/A',
  'RBBP6 0.001',
  'NFATC2IP N/A',
  'APOBR N/A',
  'IL4R N/A',
  'PALB2 N/A',
  'SULT1A2 N/A',
  'CTD-3203P2.2 N/A',
  'GSG1L N/A',
  'SBK1 N/A',
  'LCMT1 N/A'],
 ['locus 2',
  'CSTF3 N/A',
  'FBXO3 N/A',
  'SLC17A6 N/A',
  'CCDC73 N/A',
  'CAPRIN1 N/A',
  'RCN1 N/A',
  'BDNF N/A',
  'METTL15 N/A',
  'CCDC34 N/A',
  'EIF3M N/A',
  'LUZP2 N/A',
  'BBOX1 0.001',
  'CAT N/A',
  'PRRG4 N/A',
  'SLC5A12 N/A',
  'QSER1 N/A

In [54]:
import pandas as pd
import numpy as np

In [58]:
opt_pop_scores = []
for i in range(100):
    key = "Network{}".format(i+1)
    score = np.random.randint(100)
    result = [key, score]
    opt_pop_scores.append(result)
cols = ["network", "score"]
top_10 = pd.DataFrame(opt_pop_scores, columns=cols).sort_values(by=["score"], ascending=False).iloc[:10]

In [63]:

top_10.iloc[0]["network"]

'network_7'